In [6]:
import pandas as pd

In [9]:
pd.read_csv("/Users/rezadoobary/Documents/MLCorrelator2/ML-correlator/Tree_classifier_for_graphs/new_stuff/results/reports/results_20251006_224933.csv")

,experiment_type,train_loops,test_loops,n_train,n_test,train_auc,test_auc,train_auc_std,test_auc_std,best_params
0,intra-loop,6,6,36,36,0.980030,0.933333,0.008235,0.058689,"{""subsample"": 0.85, ""reg_lambda"": 2.0, ""reg_al..."
1,intra-loop,7,7,220,220,0.913159,0.744117,0.042594,0.053652,"{""subsample"": 1.0, ""reg_lambda"": 1.0, ""reg_alp..."
2,intra-loop,8,8,2709,2709,0.864150,0.815280,0.014743,0.020096,"{""subsample"": 0.7, ""reg_lambda"": 0.5, ""reg_alp..."
3,intra-loop,9,9,43017,43017,0.822136,0.798852,0.003920,0.005249,"{""subsample"": 0.85, ""reg_lambda"": 0.5, ""reg_al..."
4,mixed,6,7,36,220,0.976923,0.751291,NaN,NaN,"{""subsample"": 0.85, ""reg_lambda"": 2.0, ""reg_al..."
5,mixed,7,8,220,2709,0.939844,0.775218,NaN,NaN,"{""subsample"": 1.0, ""reg_lambda"": 1.0, ""reg_alp..."
6,mixed,8,9,2709,43017,0.881142,0.761015,NaN,NaN,"{""subsample"": 0.85, ""reg_lambda"": 2.0, ""reg_al..."
7,mixed,6,8,36,2709,0.976923,0.682171,NaN,NaN,"{""subsample"": 0.85, ""reg_lambda"": 2.0, ""reg_al..."
8,mixed,6,9,36,43017,0.976923,0.660049,NaN,NaN,"{""subsample"": 0.85, ""reg_lambda"": 2.0, ""reg_al..."
9,mixed,7,9,220,43017,0.939844,0.727338,NaN,NaN,"{""subsample"": 1.0, ""reg_lambda"": 1.0, ""reg_alp..."
